# Load OOF predictions

In [124]:
import joblib
import glob

# cat_model_paths = glob.glob('models/catboost_iso_*_10_et_v4_1.41421356237-random-false_15s_cfg3_seed*_v2_r1-10_aug_gaw033_drop/0.p')
# cat_model_paths = glob.glob('models/catboost_iso_*_10_et_v4_0.6-random-true_15s_cfg3_seed*_v2_r1-10_aug_gaw033_drop/0.p')
cat_model_paths = glob.glob('models/catboost_iso_*_10_et_v6_w25_0.6-random-true_15s_cfg3_seed*_v2_r1-10_aug_gaw033_reann_drop/0.p')
cat_models = [joblib.load(p) for p in cat_model_paths]
print(len(cat_models))
print(cat_models[0]['base_oof_preds'].shape)
print(cat_models[0]['isotonic_oof_preds'].shape)

3
(233234,)
(233234,)


In [139]:
# lgbm_model_paths = glob.glob('models/lgbm_iso_*_10_et_v4_1.41421356237-random-false_15s_cfg4_seed*_r1-10_aug_gaw33/0.p')
# lgbm_model_paths = glob.glob('models/lgbm_iso_*_10_et_v4_0.6-random-true_15s_cfg5_seed*_r1-10_aug_gaw33_drop/0.p')
# lgbm_model_paths = glob.glob('models/lgbm_iso_*_10_et_v6_w100_0.6-random-true_15s_cfg5_seed*_r1-10_aug_gaw33_reann-v2_drop/0.p')
lgbm_model_paths = glob.glob('models/lgbm_iso_*_10_et_v6_w100_0.6-random-true_15s_cfg5_seed*_r1-10_aug_gaw33_drop/0.p')
lgbm_models = [joblib.load(p) for p in lgbm_model_paths]
print(len(lgbm_models))
print(lgbm_models[0]['base_oof_preds'].shape)
print(lgbm_models[0]['isotonic_oof_preds'].shape)

3
(233234,)
(233234,)


In [126]:
# tabm_model_paths = glob.glob('models/tabm_iso_*_10_et_v4_141421356237-random-false_15s_cfg2_seed*_r1-10_gaw0330/0.pkl')
# tabm_model_paths = glob.glob('models/tabm_iso_*_10_et_v4_06-random-true_15s_cfg8_seed*_r1-10_gaw0330_drop/0.pkl')
tabm_model_paths = glob.glob('models/tabm_iso_*_10_et_v6_w50_06-random-true_15s_cfg8_seed*_r1-10_gaw0330_reann_drop/0.pkl')
tabm_models = [joblib.load(p) for p in tabm_model_paths]
print(len(tabm_models))
print(tabm_models[0]['base_oof_preds'].shape)
print(tabm_models[0]['isotonic_oof_preds'].shape)

3
(233234,)
(233234,)


# Load ground truth labels

In [86]:
import polars as pl

train_test_df = pl.read_csv('/mnt/data01/data/TreeSearch/data/from_organizers/train.csv')

In [87]:
y = train_test_df['utility_agent1']

# Check baseline scores

In [140]:
from sklearn.metrics import root_mean_squared_error

base_rmses = []
iso_rmses = []
for cat_model in cat_models:
    base_rmse = root_mean_squared_error(y, cat_model['base_oof_preds'])
    iso_rmse = root_mean_squared_error(y, cat_model['isotonic_oof_preds'])

    base_rmses.append(base_rmse)
    iso_rmses.append(iso_rmse)
    print(base_rmse, iso_rmse)

print('\nMean base rmse', sum(base_rmses) / len(base_rmses))
print('Mean iso rmse', sum(iso_rmses) / len(iso_rmses))

0.36862095108952814 0.3649665122763637
0.3643488846308908 0.3597684352642133
0.3653660952363526 0.36144343638540893

Mean base rmse 0.3661119769855905
Mean iso rmse 0.362059461308662


In [141]:
base_rmses = []
iso_rmses = []
for lgbm_model in lgbm_models:
    base_rmse = root_mean_squared_error(y, lgbm_model['base_oof_preds'])
    iso_rmse = root_mean_squared_error(y, lgbm_model['isotonic_oof_preds'])

    base_rmses.append(base_rmse)
    iso_rmses.append(iso_rmse)
    print(base_rmse, iso_rmse)

print('\nMean base rmse', sum(base_rmses) / len(base_rmses))
print('Mean iso rmse', sum(iso_rmses) / len(iso_rmses))

0.36094447492375115 0.35848623038505045
0.3604770595085328 0.35790418918621636
0.35847736444768297 0.3560676571862704

Mean base rmse 0.35996629962665566
Mean iso rmse 0.3574860255858458


In [142]:
base_rmses = []
iso_rmses = []
for tabm_model in tabm_models:
    base_rmse = root_mean_squared_error(y, tabm_model['base_oof_preds'])
    iso_rmse = root_mean_squared_error(y, tabm_model['isotonic_oof_preds'])

    base_rmses.append(base_rmse)
    iso_rmses.append(iso_rmse)
    print(base_rmse, iso_rmse)

print('\nMean base rmse', sum(base_rmses) / len(base_rmses))
print('Mean iso rmse', sum(iso_rmses) / len(iso_rmses))

0.3491910888647715 0.3495742284538155
0.3526643163568214 0.3526390878483679
0.35285656925430026 0.35310747571379214

Mean base rmse 0.35157065815863103
Mean iso rmse 0.3517735973386585


# Determine optimal isotonic weights

In [143]:
from scipy.optimize import minimize
import numpy as np

def GetOptimalWeights(y_true, y_preds1, y_preds2):
    def loss_func(weights):
        return root_mean_squared_error(y_true, weights[0] * y_preds1 + weights[1] * y_preds2)
    weights = minimize(loss_func, [0.5, 0.5], method='Nelder-Mead')
    return weights.x

cat_residual_weights = []
cat_merged_rmses = []
for cat_model in cat_models:
    weights = GetOptimalWeights(y, cat_model['base_oof_preds'], cat_model['isotonic_oof_preds'])
    merged_cat_preds = weights[0] * cat_model['base_oof_preds'] + weights[1] * cat_model['isotonic_oof_preds']

    merged_rmse = root_mean_squared_error(y, merged_cat_preds)
    print(weights, merged_rmse)

    cat_residual_weights.append(weights[0])
    cat_merged_rmses.append(merged_rmse)

print('\nMean residual weight:', np.mean(cat_residual_weights))
print('Mean RMSE:', np.mean(cat_merged_rmses))

[0.11323483 0.89372672] 0.3649323894717554
[0.10179036 0.90530423] 0.3597374335982144
[0.09914129 0.90726852] 0.36141769186780015

Mean residual weight: 0.10472216078836316
Mean RMSE: 0.36202917164592324


In [144]:
lgbm_residual_weights = []
lgbm_merged_rmses = []
for lgbm_model in lgbm_models:
    weights = GetOptimalWeights(y, lgbm_model['base_oof_preds'], lgbm_model['isotonic_oof_preds'])
    merged_lgbm_preds = weights[0] * lgbm_model['base_oof_preds'] + weights[1] * lgbm_model['isotonic_oof_preds']

    merged_rmse = root_mean_squared_error(y, merged_lgbm_preds)
    print(weights, merged_rmse)

    lgbm_residual_weights.append(weights[0])
    lgbm_merged_rmses.append(merged_rmse)

print('\nMean residual weight:', np.mean(lgbm_residual_weights))
print('Mean RMSE:', np.mean(lgbm_merged_rmses))

[0.12983063 0.87616893] 0.35845127584427733
[0.1482264  0.85836758] 0.3578545230175841
[0.1920827  0.81772238] 0.35599119581955846

Mean residual weight: 0.15671324329865707
Mean RMSE: 0.35743233156047327


In [145]:
tabm_residual_weights = []
tabm_merged_rmses = []
for tabm_model in tabm_models:
    weights = GetOptimalWeights(y, tabm_model['base_oof_preds'], tabm_model['isotonic_oof_preds'])
    merged_tabm_preds = weights[0] * tabm_model['base_oof_preds'] + weights[1] * tabm_model['isotonic_oof_preds']
    # merged_tabm_preds = 0.97 * tabm_model['base_oof_preds'] + 0.03 * tabm_model['isotonic_oof_preds']
    # merged_tabm_preds += np.random.normal(0, 0.105, merged_tabm_preds.shape)

    merged_rmse = root_mean_squared_error(y, merged_tabm_preds)
    print(weights, merged_rmse)

    tabm_residual_weights.append(weights[0])
    tabm_merged_rmses.append(merged_rmse)

print('\nMean residual weight:', np.mean(tabm_residual_weights))
print('Mean RMSE:', np.mean(tabm_merged_rmses))

[ 1.25818341 -0.27984265] 0.3490518315107495
[0.67938267 0.30640674] 0.3524595600126872
[ 1.47655891 -0.51602212] 0.3525155686301919

Mean residual weight: 1.1380416626933936
Mean RMSE: 0.3513423200512095


# Determine optimal cat vs. LGBM weight

In [146]:
cat_weights = []
ensemble_rmses = []
for cat_model, lgbm_model in zip(cat_models, lgbm_models):
    # cat_residual_weight = 0.12
    cat_residual_weight = 0.14
    merged_cat_preds = cat_residual_weight * cat_model['base_oof_preds'] + (1 - cat_residual_weight) * cat_model['isotonic_oof_preds']

    # lgbm_residual_weight = 0.16
    lgbm_residual_weight = 0.14
    merged_lgbm_preds = lgbm_residual_weight * lgbm_model['base_oof_preds'] + (1 - lgbm_residual_weight) * lgbm_model['isotonic_oof_preds']

    weights = GetOptimalWeights(y, merged_cat_preds, merged_lgbm_preds)
    merged_preds = weights[0] * merged_cat_preds + weights[1] * merged_lgbm_preds

    merged_rmse = root_mean_squared_error(y, merged_preds)

    print(weights, merged_rmse)

    cat_weights.append(weights[0])
    ensemble_rmses.append(merged_rmse)

print('\nMean cat weight:', np.mean(cat_weights))
print('Mean RMSE:', np.mean(ensemble_rmses))

[0.29227175 0.72360044] 0.3572084094211941
[0.47065085 0.55289338] 0.3530765604555505
[0.38439281 0.6374224 ] 0.3529554850923837

Mean cat weight: 0.382438471940547
Mean RMSE: 0.3544134849897094


# Determine optimal 3-way ensemble weight

In [147]:
def GetOptimalWeightsX3(y_true, y_preds1, y_preds2, preds_3):
    def loss_func(weights):
        return root_mean_squared_error(y_true, weights[0] * y_preds1 + weights[1] * y_preds2 + weights[2] * preds_3)
    weights = minimize(loss_func, [0.5, 0.5, 0.5], method='Nelder-Mead')
    return weights.x

all_weights = []
ensemble_rmses = []
for cat_model, lgbm_model, tabm_model in zip(cat_models, lgbm_models, tabm_models):
    # cat_residual_weight = 0.12
    # lgbm_residual_weight = 0.16
    # tabm_residual_weight = 0.97

    cat_residual_weight = 0.105
    lgbm_residual_weight = 0.137
    tabm_residual_weight = 0.98
    
    merged_lgbm_preds = lgbm_residual_weight * lgbm_model['base_oof_preds'] + (1 - lgbm_residual_weight) * lgbm_model['isotonic_oof_preds']
    merged_cat_preds = cat_residual_weight * cat_model['base_oof_preds'] + (1 - cat_residual_weight) * cat_model['isotonic_oof_preds']
    merged_tabm_preds = tabm_residual_weight * tabm_model['base_oof_preds'] + (1 - tabm_residual_weight) * tabm_model['isotonic_oof_preds']
    # merged_tabm_preds += np.random.normal(0, 0.105, size=y.shape[0])

    weights = GetOptimalWeightsX3(y, merged_lgbm_preds, merged_cat_preds, merged_tabm_preds)
    merged_preds = weights[0] * merged_lgbm_preds + weights[1] * merged_cat_preds + weights[2] * merged_tabm_preds

    merged_rmse = root_mean_squared_error(y, merged_preds)

    print(weights, merged_rmse)

    all_weights.append(weights)
    ensemble_rmses.append(merged_rmse)

print('\nMean weights:', np.mean(all_weights, axis=0))
print('Mean RMSE:', np.mean(ensemble_rmses))

[0.31778217 0.06400233 0.63163647] 0.3447577587290351
[0.27097535 0.23853281 0.5091408 ] 0.3458766815428262
[0.39890472 0.10148923 0.51357002] 0.34569282814707714

Mean weights: [0.32922075 0.13467479 0.5514491 ]
Mean RMSE: 0.3454424228063128


# Measure overall score

In [148]:
ensemble_rmses = []
for cat_model, lgbm_model in zip(cat_models, lgbm_models):
    cat_residual_weight = 0.12
    merged_cat_preds = cat_residual_weight * cat_model['base_oof_preds'] + (1 - cat_residual_weight) * cat_model['isotonic_oof_preds']

    lgbm_residual_weight = 0.16
    merged_lgbm_preds = lgbm_residual_weight * lgbm_model['base_oof_preds'] + (1 - lgbm_residual_weight) * lgbm_model['isotonic_oof_preds']

    tabm_residual_weight = 0.97
    merged_tabm_preds = tabm_residual_weight * tabm_model['base_oof_preds'] + (1 - tabm_residual_weight) * tabm_model['isotonic_oof_preds']
    # merged_tabm_preds += np.random.normal(0, 0.105, size=y.shape[0])

    # weights = [4, 2, 1] # 0.35544357913564156
    weights = [4, 1.5, 1.2]
    merged_preds = ((weights[0] * merged_cat_preds) + (weights[1] * merged_lgbm_preds) + (weights[2] * merged_tabm_preds))/sum(weights)

    merged_rmse = root_mean_squared_error(y, merged_preds)
    ensemble_rmses.append(merged_rmse)

    print(merged_rmse)

print('\nMean RMSE:', np.mean(ensemble_rmses))

0.35406452800704635
0.35018122735515594
0.3511747063906921

Mean RMSE: 0.35180682058429813
